In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
os.chdir('drive')
os.chdir('mistaken')

In [4]:
!ls -l

total 42544355
drwxr-xr-x 2 root root        4096 Mar  5 06:24 afs
drwxr-xr-x 2 root root        4096 Apr  4 04:26 afs1
-rw-r--r-- 1 root root   353725242 Apr  4 04:22 afs1.zip
-rw-r--r-- 1 root root       15069 Apr  5 02:02 analysis_data.ipynb
-rw-r--r-- 1 root root       34151 Mar  5 09:27 big_living_background.png
-rw-r--r-- 1 root root       47809 Mar  5 09:27 big_park_background.png
-rw-r--r-- 1 root root     7286409 Mar  5 06:26 caffe
-rw-r--r-- 1 root root      586640 Mar 13 16:06 concat_indices_4023239360457093510.json
drwxr-xr-x 2 root root        4096 Mar  5 06:24 convSVM
-rw-r--r-- 1 root root       40612 Apr  6 14:00 Copy of Untitled0.ipynb
-rw-r--r-- 1 root root       37302 Apr  5 02:43 Copy of Untitled0.ipynb.invalid
drwxr-xr-x 2 root root        4096 Mar  5 06:25 data
drwxr-xr-x 2 root root        4096 Mar  5 09:04 drive
drwxr-xr-x 2 root root        4096 Mar  5 09:41 egocentric_images
drwxr-xr-x 2 root root        4096 Apr  3 03:34 egocentric_images_2
-rw-r--r-- 1 root 

In [5]:
#@title
import util
import json
import matplotlib
#matplotlib.use('Agg')
import argparse

import copy
import glob
import h5py
import matplotlib.pyplot as plt
from multiprocessing import Pool
import numpy as np
from scipy.misc import imsave, imresize
from scipy.ndimage import imread
from keras.preprocessing import image

Using TensorFlow backend.


In [0]:
import os

In [0]:
# load scenes data
with open("data/scenes.json") as f:
    scenes = json.load(f)

In [11]:
!pip install tqdm
from tqdm import tqdm

    100% |████████████████████████████████| 51kB 3.8MB/s 


In [12]:
"#Frames: " + str(len(scenes))

'#Frames: 9704'

* Create person centric image


In [0]:
def compute_character_frames(frame):
    ''' Recenters and image at the character's head
    
    sceneId_frameIndex_characterName.png
    sceneId_frameIndex_characterName_flipped.png
    sceneId_frameIndex_original.png
    sceneId_frameIndex_centered.png
    
    '''
    character_vec = util.get_characters_in_frame(frame)
    scene_type = frame['scene']['sceneType']
    scene_id = frame['assignmentId']
    frame_index = frame['hitIdx']
    
    # Render original scene
    img_filename = os.path.join(TMP_PATH, '%s_%d_all.png' % (scene_id, frame_index))
    img_filename = os.path.abspath(img_filename)
    json_filename = os.path.join(TMP_PATH, '%s_%d_all.json' % (scene_id, frame_index))
    json_filename = os.path.abspath(json_filename)
    renderer = util.get_renderer(frame, img_filename, json_filename)
    scene_config = renderer.scene_config_data[scene_type]
    z_decay =  scene_config['zSizeDecay']   
    img_pad_num = scene_config['imgPadNum']

    img = imread(img_filename, mode='RGB')
    global_img = util.get_global_image(img)
    for character in character_vec:
        character_name = character['name']
        img_filename = os.path.join(EGOCENTRIC_IMAGE_FOLDER, '%s_%d_%s.png' % (scene_id, frame_index, character_name))
        pose = util.get_part_pose(character, 'Head', z_decay, img_pad_num, renderer) 
        character_img = util.center_image_at_pose(global_img, pose)
        imsave(img_filename, character_img)
        print("Image name: ", img_filename)
        try:
          image.load_img(img_filename, target_size=(224, 224))
          print("Load OK")
        except IOError as e:
          print(str(e))

In [17]:
listError = ['3ZY8KE4ISJ2KP0UCFSU628NG608QVU']
for (index, scene) in tqdm(enumerate(scenes), total = len(scenes)):
  
  if (scene['assignmentId'] in listError):
    compute_character_frames(scene)


  0%|          | 0/9704 [00:00<?, ?it/s]

 97%|█████████▋| 9409/9704 [00:31<00:00, 299.09it/s]

('Image name: ', u'egocentric_images/3ZY8KE4ISJ2KP0UCFSU628NG608QVU_0_Doll15.png')
Load OK


ValueError: ignored

In [0]:
import scipy

In [11]:
!pip install tqdm
from tqdm import tqdm

    100% |████████████████████████████████| 51kB 2.1MB/s 


In [0]:
DATA_PATH = 'data'                              # Where to store temporary files
PUBLIC_PATH = 'public'                          # Where to show graphs and diagnostics
EGOCENTRIC_IMAGE_FOLDER = 'egocentric_images'   # Where egocentric images are stored
TMP_PATH = 'tmp'  

In [0]:
pool = Pool(8)
r = list(tqdm(pool.imap(compute_character_frames, scenes), total=len(scenes)))



  0%|          | 0/9704 [00:00<?, ?it/s]

IOError: [Errno 5] Input/output error: u'/home/thachhoang2410/drive/mistaken/tmp/3STRJBFXOWQJ15SSXSAU54NM5GETKC_0_all.png'


Exception in thread Thread-7:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/local/lib/python2.7/dist-packages/tqdm/_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python2.7/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [0]:
with open('afs/csail.mit.edu/u/b/bce/TheoryOfMind/abstract_scenes_v002/site_data/abstract_scenes_v002_data_scene_config.json') as f:
    pass

In [0]:
import sys
sys.version
sys.version_info

sys.version_info(major=2, minor=7, micro=14, releaselevel='final', serial=0)

In [0]:
!ls /afs

csail.mit.edu


In [0]:
wget -P data http://people.csail.mit.edu/bce/mistaken/data/scenes.json

--2018-04-03 04:04:52--  http://people.csail.mit.edu/bce/mistaken/util.py
Resolving people.csail.mit.edu (people.csail.mit.edu)... 128.30.2.133
Connecting to people.csail.mit.edu (people.csail.mit.edu)|128.30.2.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7660 (7.5K) [text/x-python]
Saving to: ‘util.py.1’

util.py.1           100%[===================>]   7.48K  --.-KB/s    in 0s      

2018-04-03 04:04:55 (524 MB/s) - ‘util.py.1’ saved [7660/7660]



In [0]:
with open('data/name_expression_dict.json') as f:
  name_expression_dict = json.load(f)

In [0]:
name_expression_dict['3S3AMIZX3U4U1IK7JPTHCHSSY3ZDCJ_2_Doll18']

{u'expression': 6,
 u'expression_one_hot': [0, 0, 0, 0, 0, 0, 1, 0, 0],
 u'name': 3,
 u'name_one_hot': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [0]:
print("#Characters: ",len(name_expression_dict['3S3AMIZX3U4U1IK7JPTHCHSSY3ZDCJ_2_Doll18']['name_one_hot']))
print("#Facial Expression: ",len(name_expression_dict['3S3AMIZX3U4U1IK7JPTHCHSSY3ZDCJ_2_Doll18']['expression_one_hot']))


('#Characters: ', 20)
('#Facial Expression: ', 9)


In [0]:
with open('data/present_dict.json') as f:
  present_dict = json.load(f)

In [0]:
print("Present Feature: ", present_dict['3S3AMIZX3U4U1IK7JPTHCHSSY3ZDCJ_2_Doll18'])
len(present_dict['3S3AMIZX3U4U1IK7JPTHCHSSY3ZDCJ_2_Doll18'])

('Present Feature: ', [0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1])


14

In [0]:
print("Present Feature: ", present_dict['3S3AMIZX3U4U1IK7JPTHCHSSY3ZDCJ_1_Doll18'])
len(present_dict['3S3AMIZX3U4U1IK7JPTHCHSSY3ZDCJ_1_Doll18'])

('Present Feature: ', [0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1])


14

In [0]:
a = '123'

In [0]:
del a

In [18]:
len(glob.glob(os.path.join(EGOCENTRIC_IMAGE_FOLDER,'*.png')))

16603

In [0]:
import gc
gc.collect()

0

In [0]:
import scipy
scipy.__version__

'0.19.1'